# Seismic Rays
We can use Snell's Law and simple geometry to calculate the path of a ray through a layered earth model, from source to receiver. One approach is to "shoot" rays of one type (direct, multiple, etc.) from the source at different angles and see where they end up reaching the surface. We could guess and check angles manually until we find the right distance to the reciever. The computer does the ray-shooting calculation fast enough that we might as well calculate the distance for a range of angles all at once, then select the angle with the closest result.

Once we have the takeoff angle we can calculate the horizontal slowness $p$ of the ray. Then we can also determine the traveltime and solve for the wave amplitude.

We will apply the method to study observations of the 1988 Upland earthquakes reported in Dreger and Helmberger (1990). The Dreger and Helmberger (1990) paper is available to read (dreger_1.pdf) in this directory. In this paper two small earthquakes, located 44 km from the Pasadena broadband stations were studied, modeling the complete seismograms to constrain the Earth structure and better learn what is possible in terms of waveform modeling using the new (at the time) very broadband, high dynamic range seismic instrumentation. The figure below shows the location of the earthquakes and the Pasadena seismic station. In this exercise you will 1) make measurements of differential times between direct S and the first multiple S, 2) measure the relative amplitudes of the phases, and compare the results with a prescribed velocity model assigning a depth for each of the earthquakes. Then you will perturb the velocity model (layer velocity and thickness) to see if better agreement is possible.

<img src=map.png> 

Figure 1. Map showing the location of the two Upland, California earthquakes and the Pasadena broadband station.

In [ ]:
#load a few libraries
import numpy as np
import matplotlib.pyplot as plt

# Exercise 1: "Shoot rays"
We'll work on modeling the two Upland, California earthquakes shown in lecture. The receiver distance for the records was 44km. We can begin with modeling the events at a depth of 25km.

## 1.1: Define parameters of the model

Note we define the parameters here, but further down in the notebook when you get to the part to optimize the model you may want to copy lines for b1, and z1 to make it easier to iterate to a better solution.

In [ ]:
#Define Parameters
b1=2500    #layer 1 velocity m/s
rho1=2400  #layer 1 density kg/m^3
z1=4000    #depth of layer 1 m
mu1=rho1*(b1**2)

b2=3500    #layer 2 velocity m/s
rho2=2670  #layer 2 density kg/m^3
mu2=rho2*(b2**2)

b3=4500    #layer 2 velocity m/s
rho3=3300  #layer 2 density kg/m^3
z3=35000   #depth of layer 3 m
mu3=rho3*(b3**2)

h=25000     #depth of source(earthquake) in meters
x=44000    #distance of recording station in meters

## 1.2: Shooting individual rays
You can play with the angle i here to determine the arrival distance. You want to find the angle that gives the correct distance to the station (44 km). Once this angle is determined the arrival time, and amplitude for the direct wave can be determined. Review the lines of the code that calculates the arrival time and the amplitude.

*** What is the calculated parameter that is necessary to estimate arrival time and amplitude? ***

In [ ]:
#find traveltime: first we need to find the angle i that results in a ray reaching the station at desired distance

#distance test equation
i=10    #test angle of incidence in degrees
x1=(h-z1)*np.tan(i*np.pi/180)
x2=z1*np.tan(np.arcsin(b1/b2*np.sin(i*np.pi/180)))
xtest=x1 + x2
print(f' i={i:f} deg.   estimated_X={xtest/1000:.3f} km target_X={x/1000:.3f} km')

if (np.abs((x - xtest)/x) > 0.0001):
    print('try again!')

#next find slownesses
p=np.sin(i*np.pi/180)/b2
n1=np.sqrt(1/b1**2 - p**2)
n2=np.sqrt(1/b2**2 - p**2)
print(f'\nslownesses p={p:e}  n1={n1:e}   n2={n2:e} s/m')

#and traveltime
t=p*x + (h-z1)*n2 + z1*n1

#and amplitude
T=(2*mu2*n2)/(mu1*n1 + mu2*n2)
amp=1*T

print(f'Wave arrival time={t:.3f} s  amplitude={amp:.3f}')


## 1.3: Searching for rays by distance (10 pts)
Now we'll use the same equations as above over a range of angles to find the angle more efficiently. The equations for the direct wave are included. Fill in the equations for the first multiple--one reflection in the upper layer.


In [ ]:
#Reset the source depth
#h=9000     #depth of source(earthquake) in meters

#First Direct
#automatically find i
i=np.arange(0,90.0,0.0001)
xdir1=(h-z1)*np.tan(i*np.pi/180)
xdir2=z1*np.tan(np.arcsin(b1/b2*np.sin(i*np.pi/180)))
xtestdir=xdir1 + xdir2
Idir=np.argmin(np.abs(xtestdir-x))     #find index of minimum difference between xtest and target distance

#next find slownesses
p=np.sin(i[Idir]*np.pi/180)/b2
n1=np.sqrt(1/b1**2 - p**2)
n2=np.sqrt(1/b2**2 - p**2)
#and traveltime
tdir=p*xtestdir[Idir] + (h-z1)*n2 + z1*n1
#and amplitude
T=(2*mu2*n2)/(mu1*n1 + mu2*n2)
ampdir=1*T

#Next First Multiple
# BEGIN SOLUTION NO PROMPT
xmul1=...
xmul2=...

xtestmul=...                         
# END SOLUTION

Imul=np.argmin(np.abs(xtestmul-x))     #find index of minimum difference between xtest and target distance


#next find slownesses
p=np.sin(i[Imul]*np.pi/180)/b2
n1=np.sqrt(1/b1**2 - p**2)
n2=np.sqrt(1/b2**2 - p**2)
#and traveltime
tmul=...
#and amplitude
T=(2*mu2*n2)/(mu1*n1 + mu2*n2)
R=(mu1*n1 - mu2*n2)/(mu1*n1 + mu2*n2) # SOLUTION
ampmul=1*T*1*R # SOLUTION

print(f'Source depth: {h/1000} km\n')
print(f'Direct:\ni={i[Idir]:.4f} degrees\nx={xtestdir[Idir]/1000:.3f} km\nWave arrival time={tdir:.3f} s  amplitude={ampdir:.3f}\n')
print(f'Multiple:\ni={i[Imul]:.4f} degrees\nx={xtestmul[Imul]/1000:.3f} km\nWave arrival time={tmul:.3f} s  amplitude={ampmul:.3f}\n')
print(f'Diftime={tmul-tdir:.3f} s  amplitude ratio={np.abs(ampdir)/np.abs(ampmul):.3f}\n')


## 1.4: Evaluating source depth (10 pts)
For the given distance, velocity, and density parameters re-compute the direct and multiple wave characteristics for a source depth of 9km and 18km. Discuss the timing and relative amplitude of the direct wave and first multiple arrivals. How might these arrivals be used to distinguish source depth when reviewing a seismogram?


# Exercise 2: Compare to Record
The tangential component records for the two 1988 Upland California earthquakes are included in Fig. 1.
 
<img src=records_annotated2.png>  

**Figure 1**: Broadband, Wood-Anderson long period, and Wood-Anderson short period records for the 1988 Upland California earthquakes.

## 2.1: Measure records (10 pts)
Make relative time and relative amplitude measurements for the direct and multiple arrivals. To isolate the amplitudes of the direct and multiple bodywaves, we will make negative peak to positive peak measurements on the short-period Wood-Anderson (wa.sp) instrement, which effectively removes low frequency surface wave contributions to the waveform.

Figure 8 in Dreger and Helmberger (1990) shows that the amplitude of the first multiple is strongly depth dependent increasing in relative amplitude significantly as the depth of the earthquake shallows. Let's therefore use the ratio of the amplitude of direct S wave to the first multiple as a metric. 


## 2.2: Estimate Source Depth (10 pts)
Use your relative time and relative amplitude measurements to estimate the depth of the two 1988 Upland California earthquakes. How close are you able to fit the calculated relative times and amplitudes to what you measured? Remember that the calculation is set up for a source in the second layer, and therefore the shallowest depth you can consider is perhaps a 10th of a km deeper than the base of the layer (z1=4000m + 100m).

You can answer this question by testing different depths in the cells above, but a better approach is to write a loop to compute the differental times and the amplitude ratios for a suite of depths. To facilitate this I have prepared a function 'findamptime(depth, distance)' that can be called in a loop over depth. It is then possible to make a plot comparing the differential time and amplitude ratio over a range of depth from say 4100 to 16000m. Points can be added to the plot for the measured values of differential time and amplitude ratio and the inferred source depth.


In [ ]:
#This function takes depth (h in meters) and distance (x in meters) and returns the differential time and
#absolute value of the direct/multiple amplitude ratio.

#You do not need to modify this function, but do review it to understand what it is doing.

def findamptime(h,x):
    #First Direct
    #automatically find i
    i=np.arange(0,90.0,0.0001)
    xdir1=(h-z1)*np.tan(i*np.pi/180)
    xdir2=z1*np.tan(np.arcsin(b1/b2*np.sin(i*np.pi/180)))
    xtestdir=xdir1 + xdir2
    Idir=np.argmin(np.abs(xtestdir-x))     #find index of minimum difference between xtest and target distance

    #next find slownesses
    p=np.sin(i[Idir]*np.pi/180)/b2
    n1=np.sqrt(1/b1**2 - p**2)
    n2=np.sqrt(1/b2**2 - p**2)
    #and traveltime
    tdir=p*xtestdir[Idir] + (h-z1)*n2 + z1*n1
    #and amplitude
    T=(2*mu2*n2)/(mu1*n1 + mu2*n2)
    ampdir=1*T
    
    #Next First Multiple
    xmul1=(h-z1)*np.tan(i*np.pi/180)
    xmul2=z1*np.tan(np.arcsin(b1/b2*np.sin(i*np.pi/180)))
    xtestmul=xdir1 + 3*xdir2               #the factor 3 is for how many times the wave travels through the layer           
    Imul=np.argmin(np.abs(xtestmul-x))     #find index of minimum difference between xtest and target distance


    #next find slownesses
    p=np.sin(i[Imul]*np.pi/180)/b2
    n1=np.sqrt(1/b1**2 - p**2)
    n2=np.sqrt(1/b2**2 - p**2)
    #and traveltime
    tmul=p*xtestmul[Imul] + (h-z1)*n2 + 3*z1*n1   #note factor of 3 SOLUTION
    #and amplitude
    T=(2*mu2*n2)/(mu1*n1 + mu2*n2)
    R=(mu1*n1 - mu2*n2)/(mu1*n1 + mu2*n2) # SOLUTION
    ampmul=1*T*1*R # SOLUTION
    
    return (tmul-tdir), (np.abs(ampdir)/np.abs(ampmul))


In [ ]:
#Write a loop to calculate the differential time and relative amplitude and then plot the data measurements




#### Write discussion of results here

## 2.3: Improve the Fit (10 pts)
Attempt to improve the fit to the earthquakes by adjusting the upper layer shear wave velocity and/or layer thickness to better match relative timing and refine your relative depth estimates.

Based on other information (Love waves) Dreger and Helmberger argued the M3.7 and M4.6 were at depths of 6 and 9 km respectively. Assuming those depths. Adjust the velocity model to improve the fit to both the differential time and amplitude ratio data.

I suggest also using a quantitative measure of fit such as the sum of the squared difference between observation and model (both time and amplitude; (observed - model)**2)

In [ ]:
#Write a loop to calculate the differential time and relative amplitude and then plot the data measurements

#Observations - use the assumed depths for the earthquakes
m46depth=6.0
m37depth=9.0
#Redefine the Velocity model parameters to adjust
b1=...
z1=...


#### Write discussion of your results of improving the velocity model for assumed depths of 6 and 9 km for the mainshock and aftershock here.

## 2.4: Moho Reflection (10 pts)
Set up the problem and write modified code for the Moho reflection (green ray below). Based on your arrival time and amplitude results, speculate whether or not the Moho reflection is important in the Upland earthquake data.

<img src=ray_plot_example.png>

**Figure 3**: Example ray plot showing direct (red) and Moho reflection (green) rays.

#### Write your discussion of the results and whether the Moho reflection is likely important in this data at 44 km distance.

# Exercise 3: Extra Credit
The following exercise is worth 10 pts of homework credit that will be added to total homework score.

## 3.1: Plot the Rays (15 pts)
Write code to plot the velocity model and the ray paths for the direct, first multiple and Moho reflection. The above ray plot is an example.

# Submission

Save your notebook as a pdf and upload to bcourses